## References 

Lecture - 12 slides for gain and entropy equation

In [ ]:
import sys
import csv
import math
import copy
import time
import numpy as np
from collections import Counter
from numpy import genfromtxt
import pandas as pd

### Report

In this project we worked with two datasets iris and spambase. The datasets were loaded into a variable using genfromtxt then normalized and radomized. The datasets were split into 10:90 testing and trainign sets respectively. The decision trees were initialized and for 10 folds the accuracy of its predictions were calculated and averaged. Instead, of growing full trees, a stopping mechanism was implemeted that once all the labels from the datasets were the same, the tree woud stop as that decsion would be unanimous throughout. 


In [ ]:
# reading from the file using numpy genfromtxt with delimiter ','
def load_csv(file):
    X = np.genfromtxt(file,delimiter=",",dtype=str)
    return (X)

In [ ]:
#normalize the entire dataset prior to learning using min-max normalization 
def normalize(matrix):
#   transfer the data metrix to np array in float type.
    a = np.array(matrix, dtype=float)
    #print("normalizing the entire dataset:")
    #print(a)
    #print("Before normalizing")
#   apply the normalization along the 0 axis of a using the formula: (x - x_min)/(x_max - x_min)
    a = (a - a.min(axis=0))/(a.max(axis=0) - a.min(axis=0))
    return a

In [ ]:
#method to randomly shuffle the array using the numpy.random.shuffle()
def random_numpy_array(ar):
    np.random.shuffle(ar)
    return ar

In [ ]:
#Normalize the data and generate the training labels,training features, test labels and test training
def generate_set(X):
    #print(X.shape[0])
    
#     store the label X[:,-1] to Y  So what that line did is sliced the array, taking all rows (:) but keeping the second column (-1)
    Y = X[:,-1]
    
#     reshape y to (Y's length, 1)
#     store it to j
    j = Y.reshape(len(Y), 1)
    #print("J is",j)
    
#     create the new_X which exclude the label X[:,:-1]
    X = X[:, :-1]
    
#     normalize the data step
#     using our implemented function normalize()
    X = normalize(X)
    
#     add the label back to the normiazlied X
#     using np.concatenate along axis=1
    X = np.concatenate((X, j), axis = 1)
    
#     store the size of rows of the normalized X with labels
    row_size = X.shape[1]
    
#     use the 10% of the data to be the test set.
#     store the number of testing data
#     set the ending index to be the number of testing data
    end = round((X.shape[0]) * 0.1)
    
#     set the starting idex to be 0
    start = 0

#     create a list that store all features of the testing data
    test_f = []
    
#     create a list that store all labels of the testing data
    test_l = []
    
#     create a list that store all features of the training data
    train_f = []
    
#     create a list that store all labels of the training data
    train_l = []
    
#     10-fold cross-validation:
    for i in range(10):
#         store the test set for corss-validation using X[start:end,:]
        X_test = X[start:end, :]

#         get training data before the testing data X[:start, :]
        X_train_b = X[:start, :]
    
#         get training data after the testing data X[:start, :]
        X_train_a = X[end:, :]

#         form the new training set using np.concatenate
        X_train = np.concatenate((X_train_b, X_train_a), axis = 0)
    
#         get the testing set labels
        label_test = X_test[:,-1]
        
#         flattent the labels
        label_test.flatten()
    
#         get the training set labels
        label_train = X_train[:,-1]
    
#         flattent the labels
        label_train.flatten()
    
#         create the test set exclude the labels
        X_test = X_test[:, :-1]
    
#         same for the training set
        X_train = X_train[:, :-1]

#         append test data of this fold to the list
        
        test_f.append(X_test)
    
#         append test lables of this fold to the list
        test_l.append(label_test)
    
#         do the same for the training set
        train_f.append(X_train)
        train_l.append(label_train)

#         update the index pointer
        start = end
        end = end + round((X.shape[0]) * 0.1)

#     return the fold list that contain data and label for both training and testing set.
    return test_f, test_l, train_f, train_l

In [ ]:
#build a dictionary where the key is the class label and values are the features which belong to that class.
def build_dict_of_attributes_with_class_values(X,y):
#     init a dict for attributes
    dict_att = {}
    
#     init feature list.
    features =[]
    
#     for each feature in the dataset
    for i in range(X.shape[1]):
#     find all the value correspond to this feature
        values = X[:,i]
    
#     init an attribute list
        attribute = []
    
#     init the counter to 0
        count = 0
    
#         for each value in the "all the value correspond to this feature"
        for x in values: 
#             init a empty list that store the attribute value
            att_value = []
    
#             append the this value to the list
            att_value.append(x)
    
#             append the label of this value to the list
            att_value.append(y[count])
    
#             append this list to the attribute list.
            attribute.append(att_value)
    
#             increase the counter
            count += 1
    
#         add this attribute list to the dict according to the feature index
        dict_att[i] = attribute
    
#         append the feature indx to the feature list.
        features.append(i)
    
#     return the dict and feature list.
    return dict_att, features


In [ ]:
# Iterative Dichotomiser 3 entropy calculation
def entropy(y):
#     init a class frequence dict
    class_freq ={}
    
#     init the attribute entropy to 0
    entropy = 0
    
#     for each label in y:
    for i in y:
#         this is label is already in the dict, we increase its feq
        if i in class_freq:
            x = class_freq[i]
            class_freq[i] = x+1
#          else, we set the freq to 1
        else: 
            class_freq[i] = 1
#     calculate the cumulate entropy using the formula.
    for x in class_freq.values():
        entropy += -(x/len(y))*math.log(x/len(y), 2)
    
    return entropy


In [ ]:
#Class node and explanation is self explaination
class Node(object):
#     init the node with val,lchild,rchild,thea and leaf.
    def __init__(self, val, lchild, rchild,thea,leaf):
        self.root_value = val
        self.root_left = lchild
        self.root_right = rchild
        self.theta = thea
        self.leaf = leaf

#     method to identify if the node is leaf
    def is_leaf(self): 
        return self.leaf

#     method to return threshold value
    def ret_thetha(self):
        return self.theta
    
#     method return root value
    def ret_root_value(self):
        return self.root_value
    
#     method return left tree
    def ret_llist(self):
        return self.root_left

#     method return right tree
    def ret_rlist(self):
        return self.root_right

    def __repr__(self):
        return "(%r, %r, %r, %r)" %(self.root_value,self.root_left,self.root_right,self.theta)


In [ ]:
#Decision tree object
class DecisionTree(object):
#     init a variable called fea_list
    fea_list = []
    
#     init the Dtree by setting the root node to None
    def __init__(self):
        self.root_node = None

    #method to return the major class value using Counter() and .most_common()
    def cal_major_class_values(self,class_values):
        freq = Counter(class_values).most_common (1)
        return freq[0][0]

    #method to calculate best threshold value for each feature
    def cal_best_theta_value(self,ke,attri_list):
#         init a list for data
        data = []
    
#         init a list for class labes
        class_labels = []
        
#         for each attribute in the attri_list
        for i in attri_list:
#             append the data
            data.append(float(i[0]))
    
#             append the feature value.
            class_labels.append(i[1])
    
#         calculate the entropy of those feaure values
        etr = entropy(class_labels)
#         init the max info gain = 0
        max_info_gain = 0
    
#         init theta=0
        theta = 0
    
#         init a list that store the best index on the left
        best_l = []
    
#         init a list that store the best index on the right
        best_r = []
    
#         init a list that store class labels after split
        split_labels =[]
    
#         sort the data
        data.sort()
    
#         for each index of data:
        for i in range(len(data)-1):
    
#             calculate the current theta using data[i]+data[i+1])/ 2
            theta = (data[i]+data[i+1])/2
    
#             init a list that store index that less than theta
            index_less_theta = []
    
#             init a list that store value that less than theta
            value_less_theta = []
    
#             init a list that store index that greater than theta
            index_great_theta = []
    
#             init a list that store value that less than theta
            value_great_theta = []
    
#             init the counter to 0
            counter = 0
    
#             for each index and value in attri_list
            for c,j in enumerate(attri_list):
#                 if value less or equal than the current theta:
                if (float(j[0])<=theta):
#                     update the "less" list of index and value
                    index_less_theta.append(j[1])
                    value_less_theta.append(c)
                else:
#                     update the "greater" list of index and value
                    index_great_theta.append(j[1])
                    value_great_theta.append(c)

#             calculate the entropy of the "less" list
            etr_less = entropy(index_less_theta)
            
#             calculate the entropy of the "greater" list
            etr_great = entropy(index_great_theta)
            
#             calculate the info gain using the formular.
            gain = etr - (len(value_less_theta)/len(data))*etr_less - (len(value_great_theta)/len(data))*etr_great
#             if current info gain > max info gan
            if (gain > max_info_gain):
#                 update the info gain, 
#                     the theta, the best index list of right, 
#                         the best index list of left and class_labels_list_after_split
                max_info_gain = gain
                best_l = value_less_theta
                best_r = value_great_theta
                split_labels = index_less_theta + index_great_theta
#         return the max info gain, theata,the best left list,the best right list and class label after split
        return max_info_gain, theta, best_l, best_r, split_labels

    #method to select the best feature out of all the features.
    def best_feature(self,dict_rep):
#         set key value to none
        key_value = None
    
#         set best info gain to -1
        best_gain = -1
    
#         set best theta to 0
        best_theta = 0
    
#         set best left list to empty
        best_l = []
    
#         set best right list to empty
        best_r = []
    
#         set best class labels after split to empty
        best_split = []
    
#         init a result list
        result = []
    
#         for each key in dict_rep:
        for ke in dict_rep.keys():
#             using cal_best_theta_value() and store all returned values
            info_gain, theta, index_l, index_r, split_labels = self.cal_best_theta_value(ke,dict_rep[ke])
#             if info gian is greater than best info gain:
            if (info_gain > best_gain):
        
#                 update info gain, theth, key value,
#                     left list, right list, class labels after split
                best_gain = info_gain
                best_theta = theta
                best_l = index_l
                best_r = index_r
                best_split = split_labels
                key_value = ke

#         append the key value to the retrun list
        result.append(key_value)
    
#         append the theta value to the retrun list
        result.append(best_theta)
    
#         append the left list to the retrun list
        result.append(best_l)
    
#         append the right list to the retrun list
        result.append(best_r)
    
#         append the class labels to the retrun list
        result.append(best_split)
    
#         return the list.
        return result

    def get_remainder_dict(self,dict_of_everything,index_split):
        global fea_list
#         init a split dict
        split = {}
    
#         for each key "ke" in dict_of_everything:
        for ke in dict_of_everything.keys():
#             init a value list
            value = []
    
#             init a modified list
            mod = []
    
#             get the corresponding values of the key"ke" 
            val = dict_of_everything[ke]
    
#             for each value and its corresponding index of the key"ke" 
            for c,j in enumerate(val):
                
#                 if index is not in the index_split:
                if(c not in index_split):
#                     append it to the modified list and value list
                    mod.append(j)
                    value.append(j[1])

#             add this modified list to the dict
            split[ke] = mod
#         return the splited dict and val list
        return split, value

    #method to create decision tree
    def create_decision_tree(self, dict_of_everything,class_val,eta_min_val):
        global fea_list
        #if all the class labels are same, then we are set
        if len(set(class_val)) == 1:   #based on the return value, the node clas has to be called to create root nodes with no children
            #print("Leaf node for set class is",class_val[0],len(class_val))
            root_node =  Node(class_val[0], None, None, 0, True)
            return root_node
        
        #if the no class vales are less than threshold, we assign the class with max values as the class label    
        elif len(class_val) < eta_min_val:
            major_label = self.cal_major_class_values(class_val)
            root_node = Node(major_label, None, None, 0, True)
            return root_node
        
#         else:
        else:
#             using the best_feature to get best feature list
            best_features = self.best_feature(dict_of_everything)
    
#             store the node name, theta,left split, right split and class labes
            node = best_features[0]
            theta = best_features[1]
            left_split = best_features[2]
            right_split = best_features[3]
            class_labels = best_features[4]

#             call get_remainder_dict to get left tree data
            left_dict, left_val = self.get_remainder_dict(dict_of_everything,left_split)
    
#             call get_remainder_dict to get right tree data
            right_dict, right_val = self.get_remainder_dict(dict_of_everything,right_split)
    
#             call create_decision_tree to get left tree based on the left tree data
            left_node = self.create_decision_tree(left_dict,left_val,eta_min_val)
    
#             call create_decision_tree to get right tree based on therightleft tree data
            right_node = self.create_decision_tree(right_dict,right_val,eta_min_val)
    
#             set the root node
            root_node = Node(node, left_node, right_node,theta,False)
    
#             return root node
            return root_node
            
    #fit the decisin tree
    def fit(self, dict_of_everything,cl_val,features,eta_min_val):
#         set the fea_list the value of features
        global fea_list
        fea_list = features
        
#         set the root node using the function create_decision_tree()
        root_node = self.create_decision_tree(dict_of_everything, cl_val,eta_min_val)
    
        return root_node

    def classify(self,row,root):
#         init the test dict
        test = {}
    
#         add row to the dict
        for k,j in enumerate(row):
            test[k] = j
            
#         set the current node to root
        cur_node = root
    
#         while the current node is not leaf:
        while not cur_node.leaf:  
#             implement the case whether the current shoud go to the left
            if float(test[cur_node.root_value]) <= cur_node.theta: # based on the principle used in cal_best_theta_value to determine whic index list to update
                cur_node = cur_node.root_left
            else:
                cur_node = cur_node.root_right
        
#         return the calss of the current node
        return cur_node.root_value
        
    #method to the labels for the test data
    def predict(self, X, root):
#         predict using the classify() 
        predict = []    #gets a list of predicted values form each row of X base on parameters of classify method
        for i in X:
            pred = self.classify(i, root)
            predict.append(pred)

        return predict

In [ ]:
#calculating the predicited accuracy
def accuracy_for_predicted_values(test_class_names1,l):
#     init true and false count to 0
    truec = 0
    falsec = 0
    
#     for each prediction,if predict is correct then, true++ else, false++
    for i in range(len(test_class_names1)):
        if(test_class_names1[i] == l[i]):
            truec += 1
        else:
            falsec += 1
            
#     return the acc
    acc = truec/len(l)
    return acc

In [ ]:
def main(num_arr, eta_min):
    eta_min_val = round(eta_min*num_arr.shape[0])
    
    #randomly shuffle the array so that we can divide the data into test/training
    num_arr = random_numpy_array(num_arr)
    
    #divide data into test labels,test features,training labels, training features
    test_f, test_l, train_f, train_l = generate_set(num_arr)
    
#     init cumulate acc to 0
    acc_cum = 0
    #ten fold iteration 
    for i in range(10):
        #build a dictionary with class labels and respective features values belonging to that class
        dict_att, features = build_dict_of_attributes_with_class_values(train_f[i],train_l[i])
        
        #instantiate decision tree instance
        d_tree = DecisionTree()
        
        # build the decision tree model.
        root_node = d_tree.fit(dict_att,train_l[i],features,eta_min_val)

        #predict the class labels for test features
        predict = d_tree.predict(test_f[i], root_node)
        #calculate the accuracy for the predicted values
        acc = accuracy_for_predicted_values(test_l[i],predict)
        
#         add acc to cumulate acc
        acc_cum += acc
        print("Accuracy is ",acc)
    print("Accuracy across 10-cross validation for",eta_min,"is",float(acc_cum)/10)


In [ ]:
eta_min_list = [0.05,0.10,0.15,0.20]
newfile = "iris.csv"
num_arr = load_csv(newfile)
for i in eta_min_list:
    main(num_arr,i)

In [ ]:
eta_min_list = [0.05,0.10,0.15,0.20,0.25]
newfile = "spambase.csv"
num_arr = load_csv(newfile)
for i in eta_min_list:
    main(num_arr,i) 